In [45]:
import torch
torch.cuda.is_available()

True

In [46]:
from datasets import load_dataset
paragraph_data = load_dataset("csv", data_files="C:/Study/Master Thesis/expaai_model/tmp/cleaned_data_150k.csv")
# paragraph_data["train"][0]

Using custom data configuration default-f8e75d5baeb9fcf2
Reusing dataset csv (C:\Users\ranax\.cache\huggingface\datasets\csv\default-f8e75d5baeb9fcf2\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
paragraph_data_randomized = paragraph_data["train"].shuffle(seed=42)
paragraph_data_randomized_rows = paragraph_data.num_rows['train']
train_end = round(paragraph_data_randomized_rows * 0.67)

paragraph_train_dataset = paragraph_data_randomized.select([i for i in list(range(train_end))])
paragraph_test_dataset = paragraph_data_randomized.select([i for i in list(range(train_end, paragraph_data_randomized_rows))])

# paragraph_train_dataset
# paragraph_test_dataset

Loading cached shuffled indices for dataset at C:\Users\ranax\.cache\huggingface\datasets\csv\default-f8e75d5baeb9fcf2\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a\cache-20ca3d8cf98e8599.arrow
